In [198]:
YEAR = 2012
MONTH = 12
REGION = "6.2"
DEGREE = 3

In [199]:
from subprocess import call
def bash(*argv):
    call([str(arg) for arg in argv])

import pathlib
EVAL_FILE = f"../data/{YEAR}/{MONTH}/ground_sm_means_CONUS.csv"
COV_FILE = f"../data/{REGION}_topo.tif"
CROP_PATH = pathlib.Path("../code/crop_to_ecoregion.R").resolve()
ADD_COV_PATH = pathlib.Path("../code/0d-add_topos.R").resolve()

REG_EV_FILE = f"eval_{YEAR}_{MONTH}_{REGION}.csv"

print(f"{CROP_PATH} {EVAL_FILE} {REG_EV_FILE} {REGION}")
bash(CROP_PATH, EVAL_FILE, REG_EV_FILE, REGION)

print(f"{ADD_COV_PATH} {REG_EV_FILE} {COV_FILE}")
bash(ADD_COV_PATH, REG_EV_FILE, COV_FILE)

In [200]:
import pandas as pd
edf = pd.read_csv(REG_EV_FILE)

if "LSF" in edf.columns:
    edf.drop(columns="LSF", inplace=True)
edf = edf.dropna()

edf = (edf - edf.mean())/edf.std()

In [201]:
from itertools import combinations_with_replacement as cwr 

covars = list(edf.columns)[3:]
for d in range(2, DEGREE + 1):
    covar_prods = cwr(covars,d)
    for cp in covar_prods:
        new_col = "_".join(cp)
        edf[new_col] = 1.0
        for c in cp:
            edf[new_col] *= edf[c]
    #print(edf.head(3))
print(f"edf shape: {edf.shape}")

edf shape: (242, 682)


In [202]:
correlations = {col:edf["V3"].corr(edf[col]) for col in edf.columns}
print(list(correlations.items())[:18])

[('V1', -0.2921723488973703), ('V2', 0.2619380308856492), ('V3', 1.0), ('DEM', -0.07740843636686058), ('HILL', -0.07335637334235548), ('SLP', -0.30991746671551884), ('ASP', -0.08644650555679825), ('CSC', -0.026577699802636273), ('LC', -0.04553617752307456), ('CI', -0.3565909717794069), ('CD', -0.07603969221943262), ('FA', 0.10754474459206866), ('TWI', -1.522320272755543e-05), ('CNB', 0.020465785961960088), ('VDC', 0.21450642546437518), ('VD', -0.011017094154728816), ('RSP', -0.11482336464050587), ('DEM_DEM', 0.03071288468695066)]


In [203]:
sorted_cors = sorted(correlations.items(), key=lambda kv: abs(kv[1]), reverse=True)
print(sorted_cors[:25])

[('V3', 1.0), ('CI', -0.3565909717794069), ('DEM_DEM_CI', -0.30997149147593817), ('SLP', -0.30991746671551884), ('DEM_DEM_SLP', -0.2964251078968221), ('V1', -0.2921723488973703), ('SLP_SLP_SLP', -0.2779150920991993), ('SLP_SLP_CI', -0.27229682487823176), ('V2', 0.2619380308856492), ('ASP_ASP_CI', -0.25901123350842964), ('CI_TWI_TWI', -0.25245927529262585), ('SLP_CI_CI', -0.24623405779583699), ('CSC_LC_CI', -0.24363586980077778), ('CI_CI_CI', -0.22566053009486023), ('LC_LC_CI', -0.22174088200252387), ('SLP_CSC_LC', -0.2203139235876143), ('VDC', 0.21450642546437518), ('SLP_TWI_TWI', -0.21370391335918726), ('CSC_CSC_VDC', 0.20766510012748507), ('CSC_CSC_CI', -0.20746681464895167), ('DEM_DEM_VDC', 0.20719320951375073), ('SLP_SLP_RSP', -0.203464795083532), ('SLP_LC_LC', -0.2033489864302236), ('CSC_LC_VDC', 0.20173986054086798), ('SLP_LC_VD', -0.19105652760989228)]
